In [ ]:
!pip install transformers datasets torch


In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import load_dataset


In [ ]:
# Load the dataset
dataset = load_dataset("cfilt/iitb-english-hindi")

# Preview the dataset
print(dataset['train'][0])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'translation': {'en': 'Give your application an accessibility workout', 'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}}


In [ ]:
# Initialize the tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Preprocess the data
def preprocess_data(example):
    # Extract the English and Hindi sentences from the 'translation' field
    input_text = example['translation']['en']  # English text
    target_text = example['translation']['hi']  # Hindi text
    input_enc = tokenizer.encode(input_text, truncation=True, padding='max_length', max_length=128)
    target_enc = tokenizer.encode(target_text, truncation=True, padding='max_length', max_length=128)
    return {'input_ids': input_enc, 'labels': target_enc}

# Apply preprocessing
train_dataset = dataset['train'].map(preprocess_data)
val_dataset = dataset['validation'].map(preprocess_data)

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("t5-small").to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Define training arguments
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
def translate(text, max_length=50):
    input_ids = tokenizer.encode(text, return_tensors='pt').to("cuda" if torch.cuda.is_available() else "cpu")
    output_ids = model.generate(input_ids, max_length=max_length, num_beams=4, early_stopping=True)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Example translation
english_text = "I am learning."
hindi_translation = translate(english_text)
print(f"English: {english_text} -> Hindi: {hindi_translation}")


English: I am learning. -> Hindi: Lernen lern ich.


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "Helsinki-NLP/opus-mt-en-hi"  # Example model name
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def translate(text, max_length=50):
    input_ids = tokenizer.encode(text, return_tensors='pt').to("cuda" if torch.cuda.is_available() else "cpu")
    output_ids = model.generate(input_ids, max_length=max_length, num_beams=4, early_stopping=True)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Example translation
english_text = "I am learning."
hindi_translation = translate(english_text)
print(f"English: {english_text} -> Hindi: {hindi_translation}")


English: I am learning. -> Hindi: मैं सीख रहा हूँ ।
